# Use code from `analyze_sample` to identify thresholds that distinguish when laser fails to turn on

## Load functions

In [1]:
# Path to R (for rpy2)
import os
os.environ['R_HOME'] = '/home/michael/anaconda3/lib/R'

# Import classes
# Check version
from gmodetector_py import version
print('Running GMOdetector version ' + version.__version__)

from gmodetector_py import XMatrix
from gmodetector_py import Hypercube
from gmodetector_py import WeightArray
from gmodetector_py import ImageChannel
from gmodetector_py import FalseColor

# Import the one function we need to use before calling classes
from gmodetector_py import read_wavelengths

# The following are needed for this specific wrapper script only
import warnings
import argparse
import time
from datetime import datetime
import ntpath

import os # needed for basename

Running GMOdetector version 0.0.855


## Configure options

In [2]:
spectral_library_path = "/home/michael/gmodetector_py/spectral_library/"
intercept = 1
spectra_noise_threshold = 0.01
min_desired_wavelength = 500
max_desired_wavelength = 900

fluorophore_ID_vector = ["GFP", "Chl", "Diffraction"]

relu_before_plot = True

## Compute array of fluorophore weights for a given hypercube

### With laser off

In [3]:
file_path = "/media/michael/Elements_13/Laser_threshold_function/PLANTLESSlaserOFF_I5.0_F1.9_L100LED30_194918_0_0_0_Fluorescence.hdr"

In [4]:
wavelengths = read_wavelengths(file_path = file_path)

test_matrix = XMatrix(fluorophore_ID_vector = fluorophore_ID_vector,
                  spectral_library_path = spectral_library_path,
                  intercept = intercept,
                  wavelengths = wavelengths,
                  spectra_noise_threshold = spectra_noise_threshold,
                  min_desired_wavelength = min_desired_wavelength,
                  max_desired_wavelength = max_desired_wavelength)


test_cube = Hypercube(file_path,
                      min_desired_wavelength = min_desired_wavelength,
                      max_desired_wavelength = max_desired_wavelength)

weight_array = WeightArray(test_matrix=test_matrix,
                           test_cube=test_cube,
                           relu = relu_before_plot)

load mode isload_full_then_crop


/home/michael/anaconda3/lib/python3.8/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)
/home/michael/anaconda3/lib/python3.8/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)


In [5]:
import numpy as np

In [6]:
weight_array.weights.shape

(1566, 1444, 4)

#### Summary stats for whole array

In [7]:
np.max(weight_array.weights)

124.26552790911411

In [8]:
np.std(weight_array.weights)

50.59577948694527

#### Summary stats for each matrix slice

##### Intercept

In [9]:
np.max(weight_array.weights[:,:,0])

124.26552790911411

In [11]:
np.std(weight_array.weights[:,:,0])

1.5609014897310411

##### GFP

In [12]:
np.max(weight_array.weights[:,:,1])

17.50230122172923

In [13]:
np.std(weight_array.weights[:,:,1])

1.8884950840506438

##### Chlorophyll

In [13]:
np.max(weight_array.weights[:,:,2])

122.59770638619983

In [14]:
np.std(weight_array.weights[:,:,2])

1.2940365079897382

##### Diffraction 

In [16]:
np.max(weight_array.weights[:,:,3])

13.634662622291104

In [17]:
np.std(weight_array.weights[:,:,3])

1.5569386799197245

#### Check for every laser off image

In [42]:
import os
import glob
import pandas as pd

In [43]:
os.chdir("/media/michael/Elements_13/Laser_threshold_function/")

In [51]:
file_list = glob.glob("*laserOFF*.hdr")

file_list.sort()

df = pd.DataFrame(columns = ['file', 'mean_intercept', 'max_intercept', 'mean_chlorophyll', 'max_chlorophyll', 'mean_GFP', 'max_GFP',
                             'mean_diffraction', 'max_diffraction'], index = range(len(file_list)))

for i in range(len(file_list)):
    file_path = file_list[i]
    
    wavelengths = read_wavelengths(file_path = file_path)

    test_matrix = XMatrix(fluorophore_ID_vector = fluorophore_ID_vector,
                      spectral_library_path = spectral_library_path,
                      intercept = intercept,
                      wavelengths = wavelengths,
                      spectra_noise_threshold = spectra_noise_threshold,
                      min_desired_wavelength = min_desired_wavelength,
                      max_desired_wavelength = max_desired_wavelength)


    test_cube = Hypercube(file_path,
                          min_desired_wavelength = min_desired_wavelength,
                          max_desired_wavelength = max_desired_wavelength)

    weight_array = WeightArray(test_matrix=test_matrix,
                               test_cube=test_cube,
                               relu = relu_before_plot)
    
    mean_intercept = np.mean(weight_array.weights[:,:,0])
    max_intercept = np.max(weight_array.weights[:,:,0])
    mean_chlorophyll = np.mean(weight_array.weights[:,:,1])
    max_chlorophyll = np.max(weight_array.weights[:,:,1])
    mean_GFP = np.mean(weight_array.weights[:,:,2])
    max_GFP = np.max(weight_array.weights[:,:,2])
    mean_diffraction = np.mean(weight_array.weights[:,:,3])
    max_diffraction = np.max(weight_array.weights[:,:,3])
    
    df.iloc[i] = [file_path, mean_intercept, max_intercept,
                  mean_chlorophyll, max_chlorophyll, mean_GFP,
                  max_GFP, mean_diffraction, max_diffraction]

/home/michael/anaconda3/lib/python3.8/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)
/home/michael/anaconda3/lib/python3.8/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)


load mode isload_full_then_crop
load mode isload_full_then_crop


/home/michael/anaconda3/lib/python3.8/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)


load mode isload_full_then_crop


/home/michael/anaconda3/lib/python3.8/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)


...

/home/michael/anaconda3/lib/python3.8/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)


load mode isload_full_then_crop


/home/michael/anaconda3/lib/python3.8/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)


load mode isload_full_then_crop


/home/michael/anaconda3/lib/python3.8/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)


In [53]:
df_laser_off = df

### With laser on, but no samples on media

In [18]:
file_path = "/media/michael/Elements_13/Laser_threshold_function/PLANTLESSlaserON_I5.0_F1.9_L100LED30_194615_0_0_0_Fluorescence.hdr"

In [19]:
wavelengths = read_wavelengths(file_path = file_path)

test_matrix = XMatrix(fluorophore_ID_vector = fluorophore_ID_vector,
                  spectral_library_path = spectral_library_path,
                  intercept = intercept,
                  wavelengths = wavelengths,
                  spectra_noise_threshold = spectra_noise_threshold,
                  min_desired_wavelength = min_desired_wavelength,
                  max_desired_wavelength = max_desired_wavelength)


test_cube = Hypercube(file_path,
                      min_desired_wavelength = min_desired_wavelength,
                      max_desired_wavelength = max_desired_wavelength)

weight_array = WeightArray(test_matrix=test_matrix,
                           test_cube=test_cube,
                           relu = relu_before_plot)

load mode isload_full_then_crop


/home/michael/anaconda3/lib/python3.8/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)


#### Summary stats for each matrix slice

##### Intercept

In [20]:
np.max(weight_array.weights[:,:,0])

144.16810761713282

In [21]:
np.std(weight_array.weights[:,:,0])

1.5832997595357379

##### GFP

In [22]:
np.max(weight_array.weights[:,:,1])

164.5967418059667

In [23]:
np.std(weight_array.weights[:,:,1])

7.907386636074557

##### Chlorophyll

In [24]:
np.max(weight_array.weights[:,:,2])

12.312077935403781

In [25]:
np.std(weight_array.weights[:,:,2])

1.1006240258728135

##### Diffraction 

In [26]:
np.max(weight_array.weights[:,:,3])

104.80118568063536

In [27]:
np.std(weight_array.weights[:,:,3])

5.053849224638607

#### Run for all samples with laser on

In [54]:
file_list = glob.glob("*laserON*.hdr")

file_list.sort()

df = pd.DataFrame(columns = ['file', 'mean_intercept', 'max_intercept', 'mean_chlorophyll', 'max_chlorophyll', 'mean_GFP', 'max_GFP',
                             'mean_diffraction', 'max_diffraction'], index = range(len(file_list)))

for i in range(len(file_list)):
    file_path = file_list[i]
    
    wavelengths = read_wavelengths(file_path = file_path)

    test_matrix = XMatrix(fluorophore_ID_vector = fluorophore_ID_vector,
                      spectral_library_path = spectral_library_path,
                      intercept = intercept,
                      wavelengths = wavelengths,
                      spectra_noise_threshold = spectra_noise_threshold,
                      min_desired_wavelength = min_desired_wavelength,
                      max_desired_wavelength = max_desired_wavelength)


    test_cube = Hypercube(file_path,
                          min_desired_wavelength = min_desired_wavelength,
                          max_desired_wavelength = max_desired_wavelength)

    weight_array = WeightArray(test_matrix=test_matrix,
                               test_cube=test_cube,
                               relu = relu_before_plot)
    
    mean_intercept = np.mean(weight_array.weights[:,:,0])
    max_intercept = np.max(weight_array.weights[:,:,0])
    mean_chlorophyll = np.mean(weight_array.weights[:,:,1])
    max_chlorophyll = np.max(weight_array.weights[:,:,1])
    mean_GFP = np.mean(weight_array.weights[:,:,2])
    max_GFP = np.max(weight_array.weights[:,:,2])
    mean_diffraction = np.mean(weight_array.weights[:,:,3])
    max_diffraction = np.max(weight_array.weights[:,:,3])
    
    df.iloc[i] = [file_path, mean_intercept, max_intercept,
                  mean_chlorophyll, max_chlorophyll, mean_GFP,
                  max_GFP, mean_diffraction, max_diffraction]

/home/michael/anaconda3/lib/python3.8/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)
/home/michael/anaconda3/lib/python3.8/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)


load mode isload_full_then_crop
load mode isload_full_then_crop


/home/michael/anaconda3/lib/python3.8/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)


load mode isload_full_then_crop


/home/michael/anaconda3/lib/python3.8/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)


...

/home/michael/anaconda3/lib/python3.8/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)


load mode isload_full_then_crop


/home/michael/anaconda3/lib/python3.8/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)


load mode isload_full_then_crop


/home/michael/anaconda3/lib/python3.8/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)


In [55]:
df_laser_on = df

In [56]:
df_laser_on.to_csv("laser_on_stats.csv")

In [57]:
df_laser_off.to_csv("laser_off_stats.csv")

In [59]:
os.getcwd()

'/media/michael/Elements_13/Laser_threshold_function'

Good rule: If max signal for all fluorophores (exclude intercept) is under 50, the laser is off.

In [64]:
intercept = 1

In [66]:
weight_array.weights.shape[2]

4

In [61]:
weight_array_sans_intercept = weight_array.weights[:, :, intercept:weight_array.weights.shape[3]]

In [68]:
weight_array_sans_intercept.max()

129.4689030090608

In [63]:
weight_array_sans_intercept.shape

(1569, 1444)